# Spooky Author Classification - Kaggle Submission

In [1]:
import feature_engineering
from sklearn.feature_extraction.text import CountVectorizer
from topic_modeling import get_topic_probs, get_new_topic_probs
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
import pandas as pd
import csv
import warnings
warnings.filterwarnings('ignore')

# Feature Engineering

In [5]:
train_data = feature_engineering.prepare_data('train.csv')
train_df = train_data.apply(lambda x: feature_engineering.add_features(x), axis=1)
train_df['vector_avg'] = train_df['vector_avg'] - train_df['vector_avg'].min()
train_df['FleischReadingEase'] = train_df['FleischReadingEase'] - train_df['FleischReadingEase'].min()

In [4]:
test_data = feature_engineering.prepare_data('test.csv', test=True)
test_df = test_data.apply(lambda x: feature_engineering.add_features(x), axis=1)
test_df.set_index('id', inplace=True)
test_df['vector_avg'] = test_df['vector_avg'] - test_df['vector_avg'].min()
test_df['FleischReadingEase'] = test_df['FleischReadingEase'] - test_df['FleischReadingEase'].min()

## Topic Modeling

In [6]:
train_topic_probs, lda_model = get_topic_probs(train_df)
train_topic_probs.index = train_df.index
test_topic_probs = get_new_topic_probs(test_df, lda_model)
test_topic_probs.index = test_df.index


## TFIDF/Count Vecotrizer

In [9]:
cv = CountVectorizer()
cv.fit(train_df.text)
cv_train = pd.DataFrame(cv.transform(train_df.text).toarray(), index=train_df.index)
cv_test = pd.DataFrame(cv.transform(test_df.text).toarray(), index=test_df.index)

## Modeling

In [10]:
y_train = train_df['author']
train_df.drop(['text', 'lemmas', 'entities', 'author'], axis=1, inplace=True)
X_train = pd.concat([train_df, train_topic_probs, cv_train], axis=1)

In [37]:
test_topic_probs.columns = ['topic_0', 'topic_1', 'topic_2', 'topic3']

In [39]:
test_df.drop(['text', 'lemmas', 'entities'], axis=1, inplace=True)
X_test = pd.concat([test_df, test_topic_probs, cv_test], axis=1)

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
predictions = lr.predict_proba(X_test)

## Kaggle Submission File

In [46]:
with open('submission.csv', mode='w') as file:
    writer = csv.writer(file, delimiter=',')

    #way to write to csv file
    writer.writerow(['id', 'EAP', 'HPL', 'MWS'])
    for i in range(len(predictions)):
        _id = X_test.index[i]
        preds = predictions[i]
        row = [_id, preds[0], preds[1], preds[2]]
        writer.writerow(row)
    

In [2]:
import visualisations

In [3]:
df = pd.read_csv('train.csv')

In [4]:
corpus,lda_model = visualisations.create_pyLDAvis(df)

In [5]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)

In [6]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.302254 -0.105044       1        1  26.802931
2      0.001427  0.317754       2        1  25.318115
3     -0.187893 -0.092463       3        1  25.072992
0     -0.115789 -0.120247       4        1  22.805962, topic_info=     Category        Freq     Term       Total  loglift  logprob
321   Default  996.000000      man  996.000000  30.0000  30.0000
62    Default  893.000000     find  893.000000  29.0000  29.0000
20    Default  805.000000    never  805.000000  28.0000  28.0000
249   Default  848.000000     time  848.000000  27.0000  27.0000
417   Default  748.000000      old  748.000000  26.0000  26.0000
...       ...         ...      ...         ...      ...      ...
657    Topic4  241.618439     away  242.400162   1.4749  -5.3505
372    Topic4  239.579010   person  240.378204   1.4748  -5.3590
980    Topic4  238.070114  thought  238.883286   1.4747  -5.3653
1269   Topic4  212.616638     bear  213.407547   1.4744  -5.4784
1147   Topic4  213.975159   father  214.800125   1.4743  -5.4720

[152 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
170       1  0.996959      almost
2590      2  0.995788  altogether
692       3  0.994302     ancient
774       2  0.999187      appear
2782      2  0.996497         arm
...     ...       ...         ...
496       3  0.997825       world
2606      2  0.998374       write
56        1  0.995903        year
468       3  0.995691         yet
421       4  0.995960       young

[122 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 4, 1])